# Transfer Learning

The follwing items are the main points about transfer learning and this page:

1. Using the existing model
1. Fine tuning: unfreeze specific layers
1. Data augmentation: generate additional data from the given data
1. Evaluation and visualization


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models


In [5]:
# Use the existing model
#model = models.resnet18(pretrained=True).to(device)
model = models.efficientnet_b0(pretrained=True)

# Swap the output layer with the classifier for two categories
#num_features = model.fc.in_features
#model.fc = nn.Linear(num_features, 2)
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, 2)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [1]:
import tensorflow_datasets as tfds
import numpy as np

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image

# Load data
#(train_ds, test_ds), ds_info = tfds.load('cats_vs_dogs', split=['train[:0.1%]', 'train[0.1%:0.12%]'], with_info=True, as_supervised=True)
#(train_ds, test_ds), ds_info = tfds.load('cats_vs_dogs', split=['train[:80%]', 'train[80%:]'], with_info=True, as_supervised=True)
(train_ds, test_ds), ds_info = tfds.load('cats_vs_dogs', split=['train[:3000]', 'train[3000:3200]'], with_info=True, as_supervised=True)

# convert the datasets to NumPy
train_images = []
train_labels = []
for image, label in tfds.as_numpy(train_ds):
    train_images.append(Image.fromarray(image))  # PIL
    train_labels.append(label)
test_images = []
test_labels = []
for image, label in tfds.as_numpy(test_ds):
    test_images.append(Image.fromarray(image))  # PIL
    test_labels.append(label)

# Define a custom Datase class
class CatsVsDogsDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# Define the transform function (for EfficientNet)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# create dataloader objects
trainset = CatsVsDogsDataset(train_images, train_labels, transform=transform)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
testset = CatsVsDogsDataset(test_images, test_labels, transform=transform)
testloader = DataLoader(testset, batch_size=32, shuffle=True)


In [12]:
# Create the loss function and the optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [15]:
# Learning the model

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")
model = model.to(device)

for epoch in range(5):
    model.train()
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Using cuda device
Epoch 1, Loss: 0.0050
Epoch 2, Loss: 0.0008
Epoch 3, Loss: 0.0009
Epoch 4, Loss: 0.0000
Epoch 5, Loss: 0.0000


In [14]:
# Evaluation

from sklearn.metrics import accuracy_score

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in testloader:
        images = images.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1).cpu()
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

print("Accuracy:", accuracy_score(all_labels, all_preds))


Accuracy: 0.985
